In [1]:
%cd ..

a:\Phong\Crawl_buffett


In [2]:
from src.volume import GetVolume

gv = GetVolume(path_save='')
gv.getVolume(id_company=1414, return_df=True, save_file=False)

['202', '3', '６', '１', '56,745,180', '202', '2', '６', '56,745,180', '202', '3', '６', '１', '3,617,422', '202', '2', '６', '3,503,192', '202', '3', '６', '１', '53,222,909', '202', '2', '６', '１', '53,807,139', '123,456,789,999', '３', '123,456,789,999', '１', '３']
2022_Q1: [56745180, 3617422]
['202', '2', '６', '１', '56,745,180', '202', '1', '６', '56,745,180', '202', '2', '６', '１', '3,040,874', '202', '1', '６', '2,917,474', '202', '2', '６', '１', '53,807,139', '202', '1', '６', '１', '53,828,024', '123,456,789,999', '３', '123,456,789,999', '１', '３']
2021_Q1: [56745180, 3040874]
['202', '1', '６', '１', '56,745,180', '202', '0', '６', '56,745,180', '202', '1', '６', '１', '2,917,186', '202', '0', '６', '2,917,126', '202', '1', '６', '１', '53,828,024', '202', '0', '６', '１', '53,828,254', '123,456,789,999', '３', '123,456,789,999', '１', '３']
2020_Q1: [56745180, 2917186]
['202', '0', '６', '１', '56,745,180', '201', '9', '６', '56,745,180', '202', '0', '６', '１', '2,916,984', '201', '9', '６', '2,916,840', '202',

EncryptedPdfError: Input PDF is encrypted. The encryption must be removed to
perform OCR.

For information about this PDF's security use
    qpdf --show-encryption infilename

You can remove the encryption using
    qpdf --decrypt [--password=[password]] infilename
